# Cost Raster Calculations and Optimal Path

In [34]:
import requests
import zipfile
import arcpy

## Pre-processing

In [35]:
# Adding the three DEMs together to create one raster layer
arcpy.management.MosaicToNewRaster("wabasha_clip_dem;dem_1m_m_Clip_winona;olmstead_dem_clip", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", "county_dems", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "16_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3\\Lab3.gdb\\county_dems'>

## Slope DEM

In [36]:
# Calculating slope for the study area DEMs
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    out_raster = arcpy.sa.Slope("county_dems", "PERCENT_RISE", 1, "PLANAR", "METER"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Slope_county1")

In [38]:
# Rescaling by function for slope
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", compression="NONE", pyramid="NONE", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    Rescale_coun1 = arcpy.sa.RescaleByFunction("Slope_county1", "MSSMALL 0.5 0.5 0 # 1000 #", 1, 10); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Rescale_coun1")

## Land Use Data

In [39]:
# Reclassifying the land use data based on the outlined preferences
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", compression="NONE", pyramid="NONE", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.ddd.Reclassify("NLCD_2016_Land_Cover_Clip", "Value", "11 7;21 1;22 1;23 1;24 1;31 1;41 1;42 1;43 1;52 1;71 1;81 10;82 10;90 5;95 5", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Reclass_NLCD1", "DATA")

In [40]:
# Weighted Sum (both layers weighted as 1)
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    Weighte_Recl1 = arcpy.ia.WeightedSum("Reclass_NLCD1 Value 1;Rescale_coun1 VALUE 1"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Weighte_Recl1")

In [41]:
# Finding the Optimal Connection
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.sa.OptimalRegionConnections("Points_XYTableToPoint1", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Optimal_Connection", None, "Weighte_Recl1", None, "PLANAR", "GENERATE_CONNECTIONS")

## Alternate Routes

Changing weights/classifications

Optional Version 1: Landcover value 1, Rescaled slope value 0.5

In [50]:
# Changing the weights so Landcover is a 1 but Rescaled slope is 0.5
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    Weighte_Recl2 = arcpy.ia.WeightedSum("Reclass_NLCD1 Value 1;Rescale_coun1 VALUE 0.5"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Weighte_Recl2")

In [51]:
# Finding the Optimal Connection version 2
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.sa.OptimalRegionConnections("Points_XYTableToPoint1", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Optimal_Connection2", None, "Weighte_Recl2", None, "PLANAR", "GENERATE_CONNECTIONS")

Optional Version 2: Landcover value 0.5, Rescaled slope value 1

In [52]:
# Changing the weights so Landcover is 0.5 and Rescaled slope is 1
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    Weighte_Recl3 = arcpy.ia.WeightedSum("Reclass_NLCD1 Value 0.5;Rescale_coun1 VALUE 1"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Weighte_Recl3")

In [53]:
# Finding the Optimal Connection version 3
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.sa.OptimalRegionConnections("Points_XYTableToPoint1", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Optimal_Connection3", None, "Weighte_Recl3", None, "PLANAR", "GENERATE_CONNECTIONS")

Changing Land Use Classification and creating a new optimal version

In [54]:
# Reclassifying the land use data based on the outlined preferences - changing open water to a 10 cost
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.ddd.Reclassify("NLCD_2016_Land_Cover_Clip", "Value", "11 10;21 1;22 1;23 1;24 1;31 1;41 1;42 1;43 1;52 1;71 1;81 10;82 10;90 5;95 5", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Reclass_NLCD2", "DATA")

In [55]:
# Weighted Sum (both layers weighted as 1)
Weighte_Recl4 = arcpy.ia.WeightedSum("Reclass_NLCD2 Value 1;Rescale_coun1 VALUE 1"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Weighte_Recl4")

In [56]:
# Finding the Optimal Connection version 4
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"):
    arcpy.sa.OptimalRegionConnections("Points_XYTableToPoint1", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb\Optimal_Connection4", None, "Weighte_Recl4", None, "PLANAR", "GENERATE_CONNECTIONS")